Before running this application, please remove and replace all files under data/inventory and data/updates in order to execute latest changes.

data/inventory
https://admin.shoplineapp.com/admin/waddystore/reports?createdBy=pos
實體店 > 庫存總表
Retail Store > Product Inventory Report

data/updates
https://admin.shoplineapp.com/admin/waddystore/products/import/bulkupload_v2
匯出商品更新檔
Export Product Bulk Update File

In [29]:
import inv_dataset
import browser
import os
import pandas as pd
import csv

In [30]:
inventory_folder = 'data/inventory/'
udpate_folder_path = 'data/updates/'

#convert inventory xls to csv
inventory_files = inv_dataset.Converter()
inventory_files.to_csv(inventory_folder)

#convert updates xls to csv
update_files = inv_dataset.Converter()
update_files.to_csv(udpate_folder_path)

In [31]:
#read inventory csv into dataframe
inventory = inv_dataset.Reader()
inv_df = inventory.get_df(inventory_folder)

In [50]:
def PreOrderClose():
    process_list = []
    #Start update procedure
    os.chdir(udpate_folder_path)
    #go thru each bulk update form csv
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            df = pd.read_csv(file_path, index_col=None, header=0)
            df = df.drop(index=0)

            df = df.loc[df['Preorder Product'] == "Y"]
            df = df.loc[df['Status'] == "Y"]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] > 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    process_list.append([sku_id, bar, has_varient])

    os.chdir("../..") 
    print(process_list) 
    clicker = browser.Action()
    clicker.close_preorder(process_list)   
    print("All Completed, End Task.")    

- You may encounter NoSuchElementException or ElementNotInteractableException, try to run this part of code again

In [51]:
PreOrderClose()

[['5f48e9ca6b3c6a002aab2100', '7350075027802', True], ['5f4caab35b5b6200367af3e2', '4061504001777', True], ['5f78a5967985380038bc15b1', '4049369016525', False], ['602e08bd4a77920029dd48df', '9504000162191', False], ['6062fa4eec7042002f6a36f0', '7350075028182', True], ['61874a6d2a89c2001702969e', '4061504009605', False], ['621c7c62e840090026155a2e', '4061504009629', False], ['623a957b6e0b6d0011b8c21b', '4061504018362', False], ['6361f1e0ad475d0010e17538', '7350075024085', False]]
Clicker starts
Login sucessful
Now browsing to SKU: 5f48e9ca6b3c6a002aab2100
Go to Variations Tab
Checkbox Status is  None
pass clicking checkbox, already unchecked
Go to Settings Tab
No action, Switch alraedy off
Saved changes, completed
Now browsing to SKU: 5f4caab35b5b6200367af3e2
Go to Variations Tab
Checkbox Status is  None
pass clicking checkbox, already unchecked
Go to Settings Tab
No action, Switch alraedy off
Saved changes, completed
Now browsing to SKU: 5f78a5967985380038bc15b1
Go to Price and Quantit

In [103]:
def PreOrderOpen():
    file = open("search/namelist.csv", "r")
    data = list(csv.reader(file, delimiter=","))
    file.close()
    search_for = dict([[row[0], row[1]] for row in data])
    search_for.pop('Keywords')

    file = open("template/period_template.csv", "r")
    words = list(csv.reader(file, delimiter=","))
    file.close()

    process_list=[]

    #Start update procedure
    #go thru each bulk update form csv
    os.chdir(udpate_folder_path)
    for file in os.listdir():
        if file.endswith(".csv"):
            #in each bulk update form csv
            file_path = f"{file}"
            update_df = pd.read_csv(file_path, index_col=None, header=0)
            update_df = update_df.drop(index=0)

            period_type = ''

            df = update_df.loc[update_df['Product Name (Traditional Chinese)'].str.contains('|'.join(list(search_for.keys())), na = False)]
            
            df = df.loc[df['Preorder Product'] == "N"]
            df = df.loc[df['Status'] == "Y"]

            discon_tag = 'dis'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag, na=False)]

            discon_tag_manual = 'shortage'
            df = df.loc[~df['Product Tag'].str.contains(discon_tag_manual, na=False)]
            
            for barcode in df['Barcode']:
                df2 = inv_df.loc[inv_df['商品條碼'] == barcode]
                df2 = df2.loc[df2['預設倉庫'] <= 0]
                for bar in df2['商品條碼']:
                    sku_id = df[df['Barcode']== bar]['Product ID (DO NOT EDIT)'].item()
                    varient_id = df[df['Barcode']== bar]['Variant ID (DO NOT EDIT)'].item()
                    chinese_product_name = df[df['Barcode']== bar]['Product Name (Traditional Chinese)'].item()

                    for key in search_for.keys():
                        if key in chinese_product_name:
                            period_type = search_for[key]
                    
                    has_varient = True
                    if pd.isnull(varient_id):
                        has_varient = False
                    
                    process_list.append([sku_id, bar, has_varient, period_type])

    print(process_list) 
    clicker = browser.Action()
    clicker.open_preorder(process_list)   
    print("All Completed, End Task.")    

In [104]:
PreOrderOpen()

[['5f3b7bcd7bfced6dfcb9ae0f', '4049369015504', False, 'A'], ['6082927b0801a40014874e00', '4712933171868', False, 'A'], ['6088ea7e2d74dd00203da227', '4251460603621', False, 'A'], ['6123d83fe69bb162d2a82a39', '66483760', False, 'A'], ['6197ca9e59c96d00320f6473', '4571361303155', False, 'A'], ['61fd714ea5ecb5002c041c52', '96821457', True, 'A'], ['63298ee03a23e9003146876a', '98000133', True, 'A'], ['632d83cf30b73f6c3a1db6a1', '98000243', False, 'A']]
Clicker starts
Login sucessful


ValueError: too many values to unpack (expected 3)

In [102]:
'''
back to parent directory if necessery
Use this line: 
os.chdir("../..")

check current directory:
os.getcwd()
'''

os.chdir("../..")
os.getcwd()

'c:\\Users\\Waddy Store\\Documents\\Code Project\\shopline_automation'